### Cargar datos

In [ ]:
import pandas as pd
datos = pd.read_csv('titulos_procesados.csv', encoding='ISO-8859-1', delimiter=',')
datos.head()

### Regresión logística - bolsa de palabras

In [ ]:
#Usar bolsa de palabras

from sklearn.feature_extraction.text import CountVectorizer

titulox = datos["title"].tolist()
vectorizer = CountVectorizer(min_df=2)
matriz_bow = vectorizer.fit_transform(titulox)

bow = pd.DataFrame(matriz_bow.toarray(), columns=vectorizer.get_feature_names_out())
bow.head()

In [ ]:
# Dividir datos

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bow, datos["classification"], test_size = 0.1, random_state=70)

In [ ]:
# Regresión logística one-vs-rest

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

modelrl = LogisticRegression(C= 0.1, max_iter= 100, penalty= 'l2', solver= 'lbfgs', multi_class='ovr')
modelrl.fit(X_train, y_train)
y_probs = modelrl.predict_proba(X_test)
ypred = modelrl.predict(X_test)

In [ ]:
# Obtener métricas detalladas por categoría

from sklearn.metrics import precision_score, classification_report
precision_por_clase = precision_score(y_test, ypred, average=None)

reporte_clasificacion = classification_report(y_test, ypred)
print(reporte_clasificacion)

In [ ]:
#Matriz de resultados

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
clases =  ['agricultural systems', 'energy systems', 'financial engineering systems', 'health systems', 'production systems', 'sustainable systems','transportation systems', 'urban systems']

# Calcula la matriz de confusión
confusion = confusion_matrix(y_test, ypred)

# Visualiza la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Etiquetas Predichas')
plt.ylabel('Etiquetas Reales')
plt.show()

### Cambiar el threshold

In [ ]:
# Ver el comportamiento del accuracy conforme se va cambiando el threshold

import numpy as np
import matplotlib.pyplot as plt

threshold = []
accuracy = []
clases = ['agricultural systems', 'energy systems', 'financial engineering systems', 'health systems', 'production systems', 'sustainable systems', 'transportation systems', 'urban systems']

for k in np.linspace(0.1, 0.9, num=30): #Recorro los valores del threshold (0,1)
  yhat_threshold = []
  resultados = 0
  for indice, prob in enumerate(y_probs): #Recorro las probabilidades obtenidas para cada observación
      classes_above_threshold = np.where(prob > k*np.max(prob))[0] #Agrego la nueva regla de decisión
      if len(classes_above_threshold) == 0:
          classes_above_threshold = [np.argmax(prob)]

      #Restricción para admitir máximo dos categorías
      if len(classes_above_threshold) > 2:
        sorted_indices = np.argsort(prob)[::-1]  # Ordena los índices por probabilidad en orden descendente
        classes_above_threshold = sorted_indices[:2]

      yhat_threshold.append(classes_above_threshold)
      cat_correcta = y_test[indice] 
      convers = clases.index(cat_correcta) 
      for i in classes_above_threshold: #Comparar si mi categoría real está dentro de las categorías seleccionadas
        if i == convers:
          resultados += 1
          break

  threshold.append(k)
  accuracy.append(resultados/2561)

# Graficar la relación entre el umbral y la precisión
plt.plot(threshold, accuracy)
plt.title('Relación entre el threshold y la exactitud del modelo')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Ver la cantidad de observaciones que están asignadas a cierto número de categorías

threshold = []
uno = []
dos = []
tres = []
cuatro = []
cinco = []
seis = []
siete = []
ocho = []

for k in np.linspace(0.1, 0.9, num=30):
  yhat_threshold = []
  resultados = 0
  for indice, prob in enumerate(y_probs): #Recorro las probabilidades obtenidas para cada observación
      classes_above_threshold = np.where(prob > k*np.max(prob))[0] #Agrego la nueva regla de decisión
      if len(classes_above_threshold) == 0:
          classes_above_threshold = [np.argmax(prob)]

      #Restricción para admitir dos categorías
      if len(classes_above_threshold) > 2:
        sorted_indices = np.argsort(prob)[::-1]  # Ordena los índices por probabilidad en orden descendente
        classes_above_threshold = sorted_indices[:2]
      yhat_threshold.append(classes_above_threshold)

  un=0
  do=0
  tr=0
  cu=0
  ci=0
  se=0
  si=0
  oc=0
  for i in yhat_threshold: #Cuento el total de observaciones que tienen cierto número de categorías asignadas
    if len(i)==1:
      un+=1
    elif len(i) == 2:
      do+=1
    elif len(i) ==3:
      tr+=1
    elif len(i) ==4:
      cu+=1
    elif len(i) ==5:
      ci+=1
    elif len(i) ==6:
      se+=1
    elif len(i) ==7:
      si+=1
    else:
      oc+=1

  uno.append(un)
  dos.append(do)
  tres.append(tr)
  cuatro.append(cu)
  cinco.append(ci)
  seis.append(se)
  siete.append(si)
  ocho.append(oc)

  threshold.append(k)

# Graficar
plt.figure(figsize=(8, 6)) 
plt.plot(threshold, uno, label='Una categorías')
plt.plot(threshold, dos, label='Dos categorías')
#plt.plot(threshold, tres, label='Tres categorías')
#plt.plot(threshold, cuatro, label='Cuatro categorías')
#plt.plot(threshold, cinco, label='Cinco categorías')
#plt.plot(threshold, seis, label='Seis categorías')
#plt.plot(threshold, siete, label='Siete categorías')
#plt.plot(threshold, ocho, label='Ocho categorías')

plt.xlabel('Threshold')
plt.ylabel('Total de observaciones')
plt.legend()  
plt.title('')
plt.show()

### Configuración del modelo final

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

clases = ['agricultural systems', 'energy systems', 'financial engineering systems', 'health systems', 'production systems', 'sustainable systems', 'transportation systems', 'urban systems']

agric = 0
energ = 0
financ = 0
health = 0
product = 0
sustai = 0
transp = 0
urban = 0

k_especifico = 0.8  #Valor del threshold elegido

# Inicializa una lista vacía para almacenar las clases predichas
yhat_threshold = []
resultados = 0

for indice, prob in enumerate(y_probs): #Recorro las probabilidades obtenidas para cada observación
      classes_above_threshold = np.where(prob > k*np.max(prob))[0] #Agrego la nueva regla de decisión
    if len(classes_above_threshold) == 0:
        classes_above_threshold = [np.argmax(prob)]

    #Si hay más de 2 categorías elegidas, selecciona aquellas dos con más probabilidad
    if len(classes_above_threshold) > 2:
        sorted_indices = np.argsort(prob)[::-1]  # Ordena los índices por probabilidad en orden descendente
        classes_above_threshold = sorted_indices[:2]

    yhat_threshold.append(classes_above_threshold)
    cat_correcta = y2_test[indice]
    convers = clases.index(cat_correcta)
    for i in classes_above_threshold:
        if i == convers:
            resultados += 1 # Toma el total de observaciones que tenían dentro de las categorías seleccionadas la categoría real
            if i == 0:
              agric += 1
            elif i == 1:
              energ += 1
            elif i == 2:
              financ += 1
            elif i == 3:
              health += 1
            elif i == 4:
              product += 1
            elif i == 5:
              sustai += 1
            elif i == 6:
              transp += 1
            else:
              urban += 1
            break

accuracy = resultados / len(y_test)

print(f"Accuracy para el threshold{k_especifico}: {accuracy}")

In [ ]:
# Obtener el accuracy por categoría

frecuencia_palabras = {}

for palabra in y_test:
    if palabra in frecuencia_palabras:
        frecuencia_palabras[palabra] += 1
    else:
        frecuencia_palabras[palabra] = 1

for palabra, frecuencia in frecuencia_palabras.items():
    print(f"La categoría '{palabra}' aparece {frecuencia} veces.")
    
# Accuracy de cada categoría
print(agric/313, energ/340, financ/347, health/290, product/291, sustai/309, transp/344, urban/327)

### Influencia

In [ ]:
#Obtener las palabras con más influencia para escoger una categoría

coefficients = modelrl.coef_
vocab = vectorizer.get_feature_names_out() #extraer el vocabulario usado
palabras = dict(zip(vocab, coefficients[7]))  #de acuerdo con la categoría de interés, en coefficients[k] k corresponde al índice correspondiente (usar la lista 'clases' como referencia)

#Influencia para que la categoría sea seleccionada
palabras_posit = sorted(palabras.items(), key=lambda x: x[1], reverse=True) # Ordenar las palabras por sus coeficientes orden descendente
top_n = 10  # Especifica la cantidad de palabras principales que se van a ver
print(f"Las {top_n} palabras con más influencia para que la categoría sea seleccionada son:")
for word, coef in palabras_posit[:top_n]:
    print(f"{word}: {coef}")
    

#Influencia para que la categoría NO sea seleccionada
palabras_negat = sorted(palabras.items(), key=lambda x: x[1], reverse=False) # Ordenar las palabras por sus coeficientes orden ascendente
top_n = 10  # Especifica la cantidad de palabras principales que se van a ver
print(f"Las {top_n} palabras con más influencia para que la categoría NO sea seleccionada son:")
for word, coef in palabras_negat[:top_n]:
    print(f"{word}: {coef}")

### Dendograma

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
datos_escalados = scaler.fit_transform(bow)
matriz_enlace = linkage(datos_escalados, method='complete', metric='euclidean')

# Visualización del dendrograma
plt.figure(figsize=(12, 8))
dendrogram(matriz_enlace, truncate_mode='level', p=30)  
plt.title('Dendrograma de Clustering Jerárquico')
plt.xlabel('Índices de observaciones')
plt.ylabel('Distancia')
plt.show()